In [ ]:
# -*- coding: utf-8 -*-
#  Copyright 2024 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck P. Vidal (UKRI-STFC)

# Register the original CAD model onto the experimental 2D radiographs

1. Load the metadata from the experiment;
2. Create a simulation environment;
    1. Start the simulation engine;
    2. Set a monochromatic source;
    3. Set the detector;
    4. Load the 3D surface of the aluminium part;
    5. Generate an X-ray image and show a visualisation of the simulation environment;
    6. Load the experimental images;
    7. Compare the simulated image with the experimental images;
3. Roughly align the 3D object;
    1. Run the optimisation;
    2. Show the evolution of the objective function;
    3. Compare the simulated image with the experimental images after registration;
4. More accurate alignment;
    1. Reduce the downsampling rate and increase the number of angles used;
    2. Compare the simulated image with the experimental images before the new registration;
    3. Perform the optimisation;
    4. Show the evolution of the objective function;
    5. Compare the simulated image with the experimental images after registration;
5. Repeat the previous optimisation;
    1. Use full resolution;
    2. Compare the simulated image with the experimental images before the new registration;
    3. Perform the optimisation;
    4. Show the evolution of the objective function;
    5. Compare the simulated image with the experimental images after registration;
6. Realistic parameters;
    1. Use the full resolution;
    2. Use a realistic beam spectrum;
    3. Add the scintillator to the detector;
    4. Compare the simulated image with the experimental images;
7. Perform a simulation of the CT scan acquisition;
    1. Apply the results of the optimisation;
    2. Perform the CT scan acquisition;
    3. Save the corresponding JSON file;
8. CT reconstruction with CIL   
    1. Load the simulated projections;
    2. Inspect the geometry;
    3. Apply the minus log;
    4. Define a ROI to save resources;
    5. Reconstruct the CT volume with CIL;
    6. Save the reconstructed CT volume to disk for example as a stack of TIFFs;
9. Voxelisation of the 3D object in the same volume space as the CT scan;
    1. Use a monochromatic beam;
    2. Use a constant for the linear attenuation coefficient of the sample, e.g. &mu; = 0.1 cm<sup>-1</sup>;
    3. Perform a CT scan acquisition in the same geometrical conditions;
10. Stop the simulation engine;   
11. CT reconstruction with CIL
    1. Load the simulated projections;
    2. Apply the minus log;
    3. Define a ROI to save resources;
    4. Reconstruct the CT volume with CIL; and
    5. Binarise the valume and save it to disk for example as a stack of TIFFs.

In [ ]:
%matplotlib inline

# Import packages
import os, math, time, sys, shutil
import numpy as np
from pathlib import Path

from tifffile import imread, imwrite
# from imageio import mimwrite
import matplotlib.pyplot as plt

import cma

from skimage.transform import pyramid_gaussian
from skimage.metrics import structural_similarity as ssim

from gvxrPython3 import gvxr
from gvxrPython3 import gvxr2json
from gvxrPython3.utils import visualise, loadSpectrumSpekpy, loadSpectrumXpecgen, has_spekpy, has_xpecgen

# import utils
# from utils import *

In [ ]:
x_default = 4*[0.5]
bounds = [4*[-1.0], 4*[1.0]]

In [ ]:
scan_string = "Wire-Cu-2mm-17.54umvx"

data_path = os.path.join("/run/media/fpvidal/DATA/CT/2023/DTHE", scan_string);
projection_path = os.path.join(data_path, "Proj");

In [ ]:
# Where the data will be saved
registration_dir = "../results/wire-registration";

# Create the directory
if not os.path.exists(registration_dir):
    os.mkdir(registration_dir);

## 1. Load the metadata from the experiment

We want to replicate the real experiment by simulation. The data extracted from the experiment parameters will be used to set the simulation.

In [ ]:
from DTHEDataReader import *
reader = DTHEDataReader(directory_path=data_path);

In [ ]:
geometry = reader.get_geometry();
print(geometry);

In [ ]:
volume_centre_position = geometry.config.system.volume_centre_position;
source_position_set = geometry.config.system.source.position_set;
detector_position_set = geometry.config.system.detector.position_set;
detector_direction_x_set = geometry.config.system.detector.direction_x_set;
detector_direction_y_set = geometry.config.system.detector.direction_y_set;

In [ ]:
image_geometry = geometry.get_ImageGeometry();
print(image_geometry)

In [ ]:
print(geometry.config.panel)

In [ ]:
experimental_img_paths = reader.projection_fname_set;

## 2. Create a simulation environment

### 2.1 Start the simulation engine

In [ ]:
gvxr.createOpenGLContext();

### 2.2 Set a monochromatic source

We want to align the object on the real radiographs. Having a monochromatic source will speed-up the process.

In [ ]:
# Set its spectrum, here a monochromatic beam
# 15000 photons of 88 keV per ray
gvxr.setMonoChromatic(98, "keV", 15000);
    
gvxr.usePointSource();
gvxr.setSourcePosition(source_position_set[0][0], source_position_set[0][1], source_position_set[0][2], "mm");

### 2.3 Set the detector

In [ ]:
gvxr.autoAlignDetector(False);
gvxr.setDetectorPosition(detector_position_set[0][0], detector_position_set[0][1], detector_position_set[0][2], "mm");
gvxr.setDetectorUpVector(detector_direction_y_set[0][0], detector_direction_y_set[0][1], detector_direction_y_set[0][2]);
gvxr.setDetectorRightVector(detector_direction_x_set[0][0], detector_direction_x_set[0][1], detector_direction_x_set[0][2]);

# To speed up the alignment, we can downsample the images:
# let pixels to compute in the simulated images, and let pixels to compare with the experimental images
downscale = 8;
gvxr.setDetectorNumberOfPixels(round(geometry.config.panel.num_pixels[0] / downscale), round(geometry.config.panel.num_pixels[1] / downscale));
gvxr.setDetectorPixelSize(geometry.config.panel.pixel_size[0] * downscale, geometry.config.panel.pixel_size[1] * downscale, "mm");

### 2.4 Load the 3D surface of the aluminium part

In [ ]:
# Where the files are
mesh_dir = "../data"

# Make sure we start we a clean environment (useful when debugging)
gvxr.removePolygonMeshesFromSceneGraph();

# Select which STL files to use
gvxr.loadMeshFile("cable_sleeve", "../data/cable_sleeve.stl", "mm", True, "root")
gvxr.loadMeshFile("copper_wires", "../data/copper_wires.stl", "mm", True, "cable_sleeve")
gvxr.invertNormalVectors("cable_sleeve")
gvxr.invertNormalVectors("copper_wires")

# Set the default material properties
default_copper_density = 8.96
default_cable_insulation_density = (1.1 + 1.35) / 2 # Flexible PVC

gvxr.setCompound("cable_sleeve", "C3H3Cl3") # PVC
gvxr.setDensity("cable_sleeve", default_cable_insulation_density, "g/cm3")
gvxr.setColour("cable_sleeve", 0.83, 0.83, 0.83, 0.2)

gvxr.setCompound("copper_wires", "Cu") # Copper
gvxr.setDensity("copper_wires", default_copper_density, "g/cm3")
gvxr.setColour("copper_wires", 0.722, 0.451, 0.2, 1)

bbox_in_mm = gvxr.getNodeAndChildrenBoundingBox("cable_sleeve", "mm")


gvxr.translateNode("cable_sleeve",
                    -bbox_in_mm[0] - (bbox_in_mm[3] - bbox_in_mm[0]) / 2.0,
                    -bbox_in_mm[1] - (bbox_in_mm[4] - bbox_in_mm[1]) / 2.0,
                    -bbox_in_mm[2] - (bbox_in_mm[5] - bbox_in_mm[2]) / 2.0,
                    "mm"
                    );

gvxr.rotateNode("cable_sleeve", 300, 0, 0);

gvxr.translateNode("cable_sleeve",
                   volume_centre_position[0],
                   volume_centre_position[1],
                   volume_centre_position[2],
                   "mm");

# gvxr.applyCurrentLocalTransformation("cable_sleeve");

In [ ]:
# gvxr.renderLoop()

### 2.5 Generate an X-ray image and show a visualisation of the simulation environment

In [ ]:
# Compute an X-ray image
# We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.
x_ray_image = np.array(gvxr.computeXRayImage()).astype(np.single);

# Update the visualisation window
gvxr.displayScene();
gvxr.setZoom(1130);
gvxr.setSceneRotationMatrix((-0.5052807927131653, 0.007959646172821522, 0.862915575504303, 0.0,
                             0.86295086145401, 0.0018172189593315125, 0.5052816271781921, 0.0,
                             0.002453952096402645, 0.9999664425849915, -0.0077874502167105675, 0.0,
                             0.0, 0.0, 0.0, 1.0));
gvxr.displayScene();

# Take a screenshot
screenshot = gvxr.takeScreenshot();

# Display the corresponding image
plt.figure();
plt.imshow(screenshot);
plt.show();

In [ ]:
plt.figure()
plt.imshow(x_ray_image, cmap="gray")

### 2.6 Load the experimental images

Downsampling is used, and 3 angles only are used.

In [ ]:
def getReference(experimental_img_paths, number_of_angles, downscale: int=1):

    images = []
    indices = []

    for i in range(number_of_angles):

        if number_of_angles == 1:
            index = 0
        else:
            # index = round((i + 1) / number_of_angles * (len(experimental_img_paths) // 2))
            index = round((i + 1) / number_of_angles * (len(experimental_img_paths) - 1))

        projection = imread(experimental_img_paths[index]).astype(np.single)

        if downscale <= 1:
            images.append(projection)
        else:
            pyramid = tuple(pyramid_gaussian(projection, downscale=downscale, channel_axis=None))
            images.append(pyramid[1])

        indices.append(index)

    ncols = images[-1].shape[1]
    images = np.array(images) / np.max(images)

    images /= np.median(images[:,:,0:ncols//5])
    images *= 255
    images[images>255]=255
    return np.round(images).astype(np.uint8), np.array(indices)

In [ ]:
downscale = 8;
ref_image, indices = getReference(experimental_img_paths, 5, downscale);

selected_source_position_set = [];
selected_detector_position_set = [];
selected_detector_direction_x_set = [];
selected_detector_direction_y_set = [];

for i in indices:
    selected_source_position_set.append(source_position_set[i]);
    selected_detector_position_set.append(detector_position_set[i]);
    selected_detector_direction_x_set.append(detector_direction_x_set[i]);
    selected_detector_direction_y_set.append(detector_direction_y_set[i]);

### 2.7 Compare the simulated image with the experimental images

It allows us to make sure that the object is well oriented in the simulation and that the rotation is performed in the same way as in the experiment.

In [ ]:
DPI = 300
def displayResult(x, figsize=(15, 4), fname=None, crop=False):
    global screenshot, bbox, DPI
    test_image = getXrayImage(x, True)

    roi_size = [ref_image.shape[1], ref_image.shape[2]]
    offset = [0, 0]

    if crop:

        roi_size[0] = round(0.9 * roi_size[0])
        roi_size[1] = round(0.7 * roi_size[1])

        offset = [
            (ref_image.shape[1] - roi_size[0]) // 2,
            (ref_image.shape[2] - roi_size[1]) // 2
        ]

    ref_tmp = np.array(ref_image, dtype=np.single)
    test_tmp = np.array(test_image, dtype=np.single)

    MAE = 0.0
    RMSE = 0.0
    SSIM = 0.0
    ZNCC = 0.0

    SSIM = 0.0

    for img1, img2 in zip(ref_tmp, test_tmp):

        tmp1 = img1[offset[0]:roi_size[0], offset[1]:roi_size[1]]
        tmp2 = img2[offset[0]:roi_size[0], offset[1]:roi_size[1]]

        MAE += compareMAE(tmp1, tmp2);
        RMSE += math.sqrt(compareMSE(tmp1, tmp2));
        SSIM += ssim(tmp1, tmp2, data_range=(tmp1.max() - tmp1.min()));

        tmp1 -= tmp1.mean()
        tmp1 /= tmp1.std()

        tmp2 -= tmp2.mean()
        tmp2 /= tmp2.std()

        ZNCC += 100 * (tmp1 * tmp2).mean()

    MAE /= ref_image.shape[0]
    RMSE /= ref_image.shape[0]
    SSIM /= ref_image.shape[0]
    ZNCC /= ref_image.shape[0]

    fig, axs = plt.subplots(len(ref_image), 4, figsize=figsize, squeeze=False)
    plt.suptitle("Overall ZNCC=" + "{:.2f}".format(ZNCC) + "%\n" +
                "Overall MAE=" + "{:.2f}".format(MAE) + "\n" +
                "Overall RMSE=" + "{:.2f}".format(RMSE) + "\n" +
                "Overall SSIM=" + "{:.2f}".format(SSIM))

    for index in range(len(ref_image)):
        axs[index][0].imshow(screenshot[index])
        axs[index][1].imshow(ref_image[index], cmap="gray", vmin=0, vmax=255)
        axs[index][2].imshow(test_image[index], cmap="gray", vmin=0, vmax=255)
        im = axs[index][3].imshow((ref_image[index].astype(np.int16) - test_image[index].astype(np.int16)), cmap="RdBu", vmin=-255, vmax=255)
        
        axs[index][0].set_title("Image: " + str(indices[index]))
            
        axs[index][1].set_title("Experimental radiograph")
        axs[index][2].set_title("Simulated radiograph")
        axs[index][3].set_title("Error map")

        axs[index][1].set_xlim((offset[1], offset[1] + roi_size[1]))
        axs[index][2].set_xlim((offset[1], offset[1] + roi_size[1]))
        axs[index][3].set_xlim((offset[1], offset[1] + roi_size[1]))

        axs[index][1].set_ylim((offset[0], offset[0] + roi_size[0]))
        axs[index][2].set_ylim((offset[0], offset[0] + roi_size[0]))
        axs[index][3].set_ylim((offset[0], offset[0] + roi_size[0]))


    plt.tight_layout()

    if fname is not None:
        plt.savefig(fname + '.pdf', bbox_inches = 'tight', dpi=DPI)
        plt.savefig(fname + '.png', bbox_inches = 'tight', dpi=DPI)

In [ ]:
def getXrayImage(x, take_screenshot=False):

    global screenshot, x_current, bounds
    screenshot = []

    test_image = np.zeros((len(selected_source_position_set), gvxr.getDetectorNumberOfPixels()[1], gvxr.getDetectorNumberOfPixels()[0]), dtype=np.uint8)

    total_incident_energy = gvxr.getTotalEnergyWithDetectorResponse()
    transformation = []
    
    if len(x) == 6:
        for lower_bound, upper_bound, val in zip(bounds[0], bounds[1], x):
            transformation.append(lower_bound + val * (upper_bound - lower_bound));
            
    elif len(x) == 4:
        for lower_bound, upper_bound, val in zip(bounds[0], bounds[1], x):
            transformation.append(lower_bound + val * (upper_bound - lower_bound));
            
        transformation.append(0.0)
        transformation.append(0.0)

    elif len(x) == 2:
        transformation = 6*[0]

        gvxr.setDensity("cable_sleeve", default_cable_insulation_density * x[0], "g/cm3")
        gvxr.setDensity("copper_wires", default_copper_density * x[1], "g/cm3")
        
#     elif len(x) == 4:
#         transformation = [x[0], x[1], x[2], x[3]]
#     elif len(x) == 3:
#         transformation = [x[0], x[1], x[2], 0.0, 0.0, 0.0]
#     elif len(x) == 5:
#         transformation = [x[0], x[1], x[2], x[3], x[4]]
#     elif len(x) == 6:
#         transformation = np.copy(x)
#     elif len(x) == len(default_weights):
#         transformation = np.copy(x_current)

#         backup_density = gvxr.getDensity(sample_label)

#         temp_weight = []
#         for i in range(len(default_weights)):
#             temp_weight.append(x[i] * default_weights[i])

#         gvxr.setMixture(sample_label,
#                         elements,
#                         np.array(temp_weight) / np.sum(temp_weight))

#         gvxr.setDensity(sample_label, default_density, "g/cm3")
#         # gvxr.setDensity(sample_label, x[0] * default_density, "g/cm3")

    label = "cable_sleeve"

    matrix_backup = gvxr.getLocalTransformationMatrix(label)

    # Sample position on turntable
    gvxr.translateNode(label, transformation[0], transformation[1], transformation[2], "mm")

    gvxr.rotateNode(label, transformation[3], transformation[4], transformation[5])

    for i, (source_position, detector_position, detector_direction_x, detector_direction_y) in enumerate(zip(selected_source_position_set, selected_detector_position_set, selected_detector_direction_x_set, selected_detector_direction_y_set)):

        gvxr.setSourcePosition(source_position[0], source_position[1], source_position[2], "mm");

        gvxr.autoAlignDetector(False);
        gvxr.setDetectorPosition(detector_position[0], detector_position[1], detector_position[2], "mm");
        gvxr.setDetectorUpVector(detector_direction_y[0], detector_direction_y[1], detector_direction_y[2]);
        gvxr.setDetectorRightVector(detector_direction_x[0], detector_direction_x[1], detector_direction_x[2]);

        # Compute an X-ray image
        # We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.
        xray_image = np.array(gvxr.computeXRayImage(), dtype=np.single) / total_incident_energy
        
        test_image[i] = np.round(255 * xray_image).astype(np.uint8)

        # Update the visualisation window
        if take_screenshot:

            gvxr.displayScene()
            screenshot.append(gvxr.takeScreenshot())

    gvxr.setLocalTransformationMatrix(label, matrix_backup)

            
#     if len(x) == len(default_weights) + 1:
#         gvxr.setMixture(sample_label,
#                         elements,
#                         default_weights)

#         gvxr.setDensity(sample_label, backup_density, "g/cm3")

    return test_image

In [ ]:
def compareMAE(ref, test):
    return np.abs(ref - test).mean()

def compareMSE(ref, test):
    return np.square(ref - test).mean()

def compareRMSE(ref, test):
    return math.sqrt(compareMSE(ref, test))

def compareZNCC(ref, test):
    if ref.std() < 1e-4 or test.std() < 1e-4:
        return 1e-4

    return np.mean(((ref - ref.mean()) / ref.std()) * ((test - test.mean()) / test.std()))

def compareSSIM(ref, test):

    channel_axis = None
    if len(ref.shape) == 3:
        channel_axis = 0

    return ssim(ref, test, channel_axis=channel_axis, data_range=(ref.max()-ref.min()))


In [ ]:
# best_fitness = sys.float_info.max
# fitness_set = []
# counter = 1
# best_angle = 0;

# plot_directory = os.path.join(registration_dir, "plot0")

# if os.path.isdir(plot_directory):
#     shutil.rmtree(plot_directory)

# if not os.path.exists(plot_directory):
#     os.mkdir(plot_directory)    

# for angle in range(380):
#     gvxr.setLocalTransformationMatrix("cable_sleeve", [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])
    
#     bbox_in_mm = gvxr.getNodeAndChildrenBoundingBox("cable_sleeve", "mm")


#     gvxr.translateNode("cable_sleeve",
#                         -bbox_in_mm[0] - (bbox_in_mm[3] - bbox_in_mm[0]) / 2.0,
#                         -bbox_in_mm[1] - (bbox_in_mm[4] - bbox_in_mm[1]) / 2.0,
#                         -bbox_in_mm[2] - (bbox_in_mm[5] - bbox_in_mm[2]) / 2.0,
#                         "mm"
#                         );

#     gvxr.rotateNode("cable_sleeve", angle, 0, 0);

#     gvxr.translateNode("cable_sleeve",
#                        volume_centre_position[0],
#                        volume_centre_position[1],
#                        volume_centre_position[2],
#                        "mm");


#     # gvxr.displayScene()
    
    
    



#     fitness = fitnessZNCC(x_default)
    
# #     if best_fitness > fitness:
# #         best_fitness = fitness
# #         best_angle = angle

# #         print(best_fitness, best_angle)

In [ ]:
# plt.figure()
# plt.title("Evolution of the fitness value of the best individual")
# plt.ylabel("ZNCC (in %)")
# plt.xlabel("Number of evaluations")
# plt.plot(np.array(fitness_set)[:, 0], 100.0 / np.array(fitness_set)[:, 1])
# plt.show()

In [ ]:
figsize = (15, 21);
displayResult(x_default, figsize=figsize, fname=None, crop=True);

## 3. Roughly align the 3D object

4 unknowns are optimised. We aim to align the object.

### 3.1 Run the optimisation

In [ ]:
def fitnessZNCC(x):
    global ref_image, best_fitness, fitness_set, counter, plot_directory

    test_image = getXrayImage(x)

    fitness_value = 0.0

    for ref, test in zip(ref_image, test_image):
        zncc = compareZNCC(ref, test)
        
        # Penalise negative values
        if zncc < 0.0:
            zncc = 1e-6
            
        # Turn it into a minimisation
        fitness_value += 1.0 / zncc

    fitness_value /= ref_image.shape[0]

    if best_fitness > fitness_value:
        fitness_set.append([counter, fitness_value])
        best_fitness = fitness_value
        displayResult(x, figsize, crop=True)
        plt.savefig(plot_directory + "/plot_" + str(counter).zfill(4) + ".png", dpi=DPI)
        plt.close()
    counter += 1

    return fitness_value

In [ ]:
opts = cma.CMAOptions()
opts.set('tolfun', 1e-5);
opts['tolx'] = 1e-5;
opts['bounds'] = [4*[0.0], 4*[1.0]];

bounds = np.array([[-5, -5, -5, -20], [5, 5, 5, 20]]);

x_fname       = os.path.join(registration_dir, "x_best1.dat")
fitness_fname = os.path.join(registration_dir, "fitness_set1.npz")
plot_directory = os.path.join(registration_dir, "plot1")

if not os.path.exists(x_fname) or not os.path.exists(fitness_fname):
    
    if os.path.isdir(plot_directory):
        shutil.rmtree(plot_directory)

    if not os.path.exists(plot_directory):
        os.mkdir(plot_directory)    

    best_fitness = sys.float_info.max
    fitness_set = []
    counter = 1

    start_time = time.time();
    es = cma.CMAEvolutionStrategy(x_default, 0.3, opts);
    es.optimize(fitnessZNCC);
    elapsed_time = time.time() - start_time

    x_best1 = es.result.xbest
    np.savetxt(x_fname, es.result.xbest)

    fitness_set = np.array(fitness_set)
    np.savez(fitness_fname, fitness_set)

    
else:
    x_best1 = np.loadtxt(x_fname)
    data = np.load(fitness_fname)
    lst = data.files
    fitness_set = data[lst[0]]

x_current = x_best1
current_actual_values = bounds[0] + x_current * (bounds[1] - bounds[0]);

In [ ]:
# Show the parameters after the optimisation
print(x_current)

### 3.2 Show the evolution of the objective function

In [ ]:
plt.figure()
plt.title("Evolution of the fitness value of the best individual")
plt.ylabel("ZNCC (in %)")
plt.xlabel("Number of evaluations")
plt.plot(fitness_set[:, 0], 100.0 / fitness_set[:, 1])
plt.show()

### 3.3 Compare the simulated image with the experimental images after registration

We can also compare the ZNCC, MAE, RMSE and SSIM before and after optimisation.

| Metrics | Before | After |
|-|--------|-------|
| ZNCC | 88.09% | 98.15% |
| MAE | 22.24 | 10.20 |
| RMSE | 36.75 | 15.51 |
| SSIM | 0.64 | 0.83 |

In [ ]:
displayResult(x_current, figsize=figsize, fname=None, crop=True)

## 4. More accurate alignment

6 unknowns are optimised. We aim to align

1. the object on the turntable (3 unknown), and
2. the centre of the turntable.

### 4.1 Reduce the downsampling rate and increase the number of angles used

It will take longer but the registration will be more accurate.

In [ ]:
downscale = 4;
figsize=(15,21);

# Update the detector resolution
gvxr.setDetectorNumberOfPixels(round(geometry.config.panel.num_pixels[0] / downscale), round(geometry.config.panel.num_pixels[1] / downscale));
gvxr.setDetectorPixelSize(geometry.config.panel.pixel_size[0] * downscale, geometry.config.panel.pixel_size[1] * downscale, "mm");

# Get the experimental images
ref_image, indices = getReference(experimental_img_paths, 5, downscale);


selected_source_position_set = [];
selected_detector_position_set = [];
selected_detector_direction_x_set = [];
selected_detector_direction_y_set = [];

for i in indices:
    selected_source_position_set.append(source_position_set[i]);
    selected_detector_position_set.append(detector_position_set[i]);
    selected_detector_direction_x_set.append(detector_direction_x_set[i]);
    selected_detector_direction_y_set.append(detector_direction_y_set[i]);

### 4.2 Compare the simulated image with the experimental images before the new registration

In [ ]:
displayResult(x_current, figsize=figsize, fname=None, crop=True);

In [ ]:
opts = cma.CMAOptions()
opts.set('tolfun', 1e-5);
opts['tolx'] = 1e-5;

x_default = np.array(6*[0.5])

bounds = [np.append(current_actual_values - 2, 2*[-2]), np.append(current_actual_values + 2, 2*[2])]

x_fname       = os.path.join(registration_dir, "x_best2.dat")
fitness_fname = os.path.join(registration_dir, "fitness_set2.npz")
plot_directory = os.path.join(registration_dir, "plot2")

if not os.path.exists(x_fname) or not os.path.exists(fitness_fname):
    
    if os.path.isdir(plot_directory):
        shutil.rmtree(plot_directory)

    if not os.path.exists(plot_directory):
        os.mkdir(plot_directory)    

    best_fitness = sys.float_info.max
    fitness_set = []
    counter = 1

    start_time = time.time();
    es = cma.CMAEvolutionStrategy(x_default, 0.2, opts);
    es.optimize(fitnessZNCC);
    elapsed_time = time.time() - start_time

    x_best2 = es.result.xbest
    np.savetxt(x_fname, es.result.xbest)

    fitness_set = np.array(fitness_set)
    np.savez(fitness_fname, fitness_set)

    
else:
    x_best2 = np.loadtxt(x_fname)
    data = np.load(fitness_fname)
    lst = data.files
    fitness_set = data[lst[0]]

x_current = x_best2
previous_actual_values = np.copy(current_actual_values);
current_actual_values = bounds[0] + x_current * (bounds[1] - bounds[0]);

In [ ]:
print(x_best1)
print(x_current)

In [ ]:
print(previous_actual_values)
print(current_actual_values)

### 4.4 Show the evolution of the objective function

In [ ]:
plt.figure()
plt.title("Evolution of the fitness value of the best individual")
plt.ylabel("ZNCC (in %)")
plt.xlabel("Number of evaluations")
plt.plot(fitness_set[:, 0], 100.0 / fitness_set[:, 1])
plt.show()

### 4.5 Compare the simulated image with the experimental images after registration

Again, we can compare the ZNCC, MAE, RMSE and SSIM before and after optimisation.

| Metrics | Before | After |
|-|--------|-------|
| ZNCC | 98.15% | 98.14% |
| MAE | 10.19 | 10.21 |
| RMSE | 15.49 | 15.53 |
| SSIM | 0.84 | 0.84 |

In [ ]:
displayResult(x_current, figsize=figsize, fname=None, crop=True)

## 5. Repeat the previous optimisation

### 5.1 Use full resolution

It will take much longer but the registration will be more accurate.

In [ ]:
downscale = 1;
figsize=(15,21);

# Update the detector resolution
gvxr.setDetectorNumberOfPixels(round(geometry.config.panel.num_pixels[0] / downscale), round(geometry.config.panel.num_pixels[1] / downscale));
gvxr.setDetectorPixelSize(geometry.config.panel.pixel_size[0] * downscale, geometry.config.panel.pixel_size[1] * downscale, "mm");

# Get the experimental images
ref_image, indices = getReference(experimental_img_paths, 7, downscale);


selected_source_position_set = [];
selected_detector_position_set = [];
selected_detector_direction_x_set = [];
selected_detector_direction_y_set = [];

for i in indices:
    selected_source_position_set.append(source_position_set[i]);
    selected_detector_position_set.append(detector_position_set[i]);
    selected_detector_direction_x_set.append(detector_direction_x_set[i]);
    selected_detector_direction_y_set.append(detector_direction_y_set[i]);

### 5.2 Compare the simulated image with the experimental images before the new registration

In [ ]:
displayResult(x_current, figsize=figsize, fname=None, crop=True);

### 5.3 Perform the optimisation

In [ ]:
opts = cma.CMAOptions()
opts.set('tolfun', 1e-6);
opts['tolx'] = 1e-6;

bounds = [current_actual_values - 2, current_actual_values + 2]

x_fname       = os.path.join(registration_dir, "x_best3.dat")
fitness_fname = os.path.join(registration_dir, "fitness_set3.npz")
plot_directory = os.path.join(registration_dir, "plot3")

if not os.path.exists(x_fname) or not os.path.exists(fitness_fname):
    
    if os.path.isdir(plot_directory):
        shutil.rmtree(plot_directory)

    if not os.path.exists(plot_directory):
        os.mkdir(plot_directory)    

    best_fitness = sys.float_info.max
    fitness_set = []
    counter = 1

    start_time = time.time();
    es = cma.CMAEvolutionStrategy(x_current, 1 / 10, opts);
    es.optimize(fitnessZNCC);
    elapsed_time = time.time() - start_time

    x_best3 = es.result.xbest
    np.savetxt(x_fname, es.result.xbest)

    fitness_set = np.array(fitness_set)
    np.savez(fitness_fname, fitness_set)
    
else:
    x_best3 = np.loadtxt(x_fname)
    data = np.load(fitness_fname)
    lst = data.files
    fitness_set = data[lst[0]]

x_current = x_best3
previous_actual_values = np.copy(current_actual_values);
current_actual_values = bounds[0] + x_current * (bounds[1] - bounds[0]);

In [ ]:
print("Previous values:", x_best2)
print("New values:     ", x_best3)

In [ ]:
print(previous_actual_values)
print(current_actual_values)

### 5.4 Show the evolution of the objective function

In [ ]:
plt.figure()
plt.title("Evolution of the fitness value of the best individual")
plt.ylabel("ZNCC (in %)")
plt.xlabel("Number of evaluations")
plt.plot(fitness_set[:, 0], 100.0 / fitness_set[:, 1])
plt.show()

### 5.5 Compare the simulated image with the experimental images after registration

Again, we can compare the ZNCC, MAE, RMSE and SSIM before and after optimisation.

| Metrics | Before | After |
|-|--------|-------|
| ZNCC | 98.08% | 98.07% |
| MAE | 10.48 | 10.50 |
| RMSE | 15.78 | 15.81 |
| SSIM | 0.91 | 0.91 |

In [ ]:
displayResult(x_current, figsize=figsize, fname=None, crop=True)

## 6. Realistic parameters

### 6.1 Use full resolution

In [ ]:
downscale = 1;
figsize=(15,21);

# Update the detector resolution
gvxr.setDetectorNumberOfPixels(round(geometry.config.panel.num_pixels[0] / downscale), round(geometry.config.panel.num_pixels[1] / downscale));
gvxr.setDetectorPixelSize(geometry.config.panel.pixel_size[0] * downscale, geometry.config.panel.pixel_size[1] * downscale, "mm");

# Get the experimental images
ref_image, indices = getReference(experimental_img_paths, 1, downscale);


selected_source_position_set = [];
selected_detector_position_set = [];
selected_detector_direction_x_set = [];
selected_detector_direction_y_set = [];

for i in indices:
    selected_source_position_set.append(source_position_set[i]);
    selected_detector_position_set.append(detector_position_set[i]);
    selected_detector_direction_x_set.append(detector_direction_x_set[i]);
    selected_detector_direction_y_set.append(detector_direction_y_set[i]);

### 6.2 Use a realistic beam spectrum

We favour Xpecgen as it provides a smaller number of energy bins.

In [ ]:
if has_xpecgen:
    loadSpectrumXpecgen(160, filters=[["Cu", 2]], th_in_deg=12)
elif has_spekpy:
    loadSpectrumSpekpy(160, filters=[["Cu", 2]], th_in_deg=12)

In [ ]:
def plotSpectrum(k, f, fname=None, xlim=[0,200], figsize=(20,10)):

    plt.figure(figsize=figsize)

    plt.bar(k, f / f.sum()) # Plot the spectrum
    plt.xlabel('Energy in keV')
    plt.ylabel('Probability distribution of photons per keV')
    plt.title('Photon energy distribution')

    plt.xlim(xlim)

    plt.tight_layout()

    if fname is not None:
        plt.savefig(fname + '.pdf', bbox_inches = 'tight', dpi=DPI)
        plt.savefig(fname + '.png', bbox_inches = 'tight', dpi=DPI)


In [ ]:
energy_bins = np.array(gvxr.getEnergyBins("keV"))
photon_counts = np.array(gvxr.getPhotonCountEnergyBins())

plotSpectrum(energy_bins,
             photon_counts,
             fname=None,
             xlim=[0,200],
             figsize=(20,10))

### 6.3 Add the scintillator to the detector

In [ ]:
gvxr.setScintillator("Gd2O2S DRZ-Plus", 210, "um")
# Gadox (Gd2O2S) DRZ+ : scintillator of 100mg/cm2 and 210um thickness which gives an apparent density of 4.76 g/cm3

In [ ]:
detector_response = np.array(gvxr.getEnergyResponse("keV"))
plt.figure(figsize= (20,10))
# plt.title("Detector response")
plt.plot(detector_response[:,0], detector_response[:,1])
plt.xlabel('Incident energy: E (in keV)')
plt.ylabel('Detector energy response: $\\delta$(E) (in keV)')

plt.tight_layout()

### 6.4 Compare the simulated image with the experimental images

In [ ]:
displayResult(x_current, figsize=figsize, fname=os.path.join(registration_dir, "registration_results"), crop=True);

In [ ]:
# Sample position on turntable
while len(current_actual_values) < 6:
    current_actual_values = np.append(current_actual_values, 0)

    
    
    
    
# gvxr.translateNode("copper_wires",
#                     -bbox_in_mm[0] - (bbox_in_mm[3] - bbox_in_mm[0]) / 2.0,
#                     -bbox_in_mm[1] - (bbox_in_mm[4] - bbox_in_mm[1]) / 2.0,
#                     -bbox_in_mm[2] - (bbox_in_mm[5] - bbox_in_mm[2]) / 2.0,
#                     "mm"
#                     );

# gvxr.rotateNode("copper_wires", 300, 0, 0);

# gvxr.translateNode("copper_wires",
#                    volume_centre_position[0],
#                    volume_centre_position[1],
#                    volume_centre_position[2],
#                    "mm");

gvxr.translateNode("cable_sleeve", current_actual_values[0], current_actual_values[1], current_actual_values[2], "mm")
gvxr.rotateNode("cable_sleeve", current_actual_values[3], current_actual_values[4], current_actual_values[5])

gvxr.applyCurrentLocalTransformation("cable_sleeve")



for label in ["cable_sleeve", "copper_wires"]:
    # gvxr.translateNode(label, current_actual_values[0], current_actual_values[1], current_actual_values[2], "mm")
    # gvxr.rotateNode(label, current_actual_values[3], current_actual_values[4], current_actual_values[5])

    # gvxr.applyCurrentLocalTransformation(label)

    mesh_fname = os.path.join(registration_dir, label + "-transformed.stl")
    gvxr.saveSTLfile(label, mesh_fname);
    print("New mesh saved in:", mesh_fname);

In [ ]:
figsize=(12,5)
displayResult([1, 1], figsize=figsize, crop=True);

# Optimise the densities

In [ ]:
def fitnessRMSE(x):
    global ref_image, best_fitness, fitness_set, counter, plot_directory

    test_image = getXrayImage(x)

    fitness_value = 0.0

    for ref, test in zip(ref_image, test_image):
        fitness_value += compareRMSE(ref, test)

    fitness_value /= ref_image.shape[0]

    if best_fitness > fitness_value:
        fitness_set.append([counter, fitness_value])
        best_fitness = fitness_value
        displayResult(x, figsize, crop=True)
        plt.savefig(plot_directory + "/plot_" + str(counter).zfill(4) + ".png", dpi=DPI)
        plt.close()

    counter += 1

    return fitness_value

In [ ]:
opts = cma.CMAOptions()
opts.set('tolfun', 1e-6);
opts['tolx'] = 1e-6;
opts['bounds'] = [2*[0.0], 2*[2.0]];

x_fname       = os.path.join(registration_dir, "x_best4.dat")
fitness_fname = os.path.join(registration_dir, "fitness_set4.npz")
plot_directory = os.path.join(registration_dir, "plot4")

if not os.path.exists(x_fname) or not os.path.exists(fitness_fname):
    
    if os.path.isdir(plot_directory):
        shutil.rmtree(plot_directory)

    if not os.path.exists(plot_directory):
        os.mkdir(plot_directory)    

    best_fitness = sys.float_info.max
    fitness_set = []
    counter = 1

    start_time = time.time();
    es = cma.CMAEvolutionStrategy([1, 1], 1 / 10, opts);
    es.optimize(fitnessRMSE);
    elapsed_time = time.time() - start_time

    x_best4 = es.result.xbest
    np.savetxt(x_fname, es.result.xbest)

    fitness_set = np.array(fitness_set)
    np.savez(fitness_fname, fitness_set)
    
else:
    x_best4 = np.loadtxt(x_fname)
    data = np.load(fitness_fname)
    lst = data.files
    fitness_set = data[lst[0]]

gvxr.setDensity("cable_sleeve", default_cable_insulation_density * x_best4[0], "g/cm3")
gvxr.setDensity("copper_wires", default_copper_density * x_best4[1], "g/cm3")

In [ ]:
test_default_densities = getXrayImage([1, 1])
test_optimised_densities = getXrayImage(x_best4)

fig, axs = plt.subplots(len(ref_image), 1, figsize=(12,10), squeeze=False)
plt.suptitle("Intensity profiles through the middle row")

row_id = test_default_densities.shape[1] // 2
row_size = test_default_densities.shape[2]

for index in range(len(ref_image)):
    axs[index][0].plot(ref_image[index][row_id], label="Experimental")
    axs[index][0].plot(test_default_densities[index][row_id], label="Default densities")
    axs[index][0].plot(test_optimised_densities[index][row_id], label="Optimised densities")
    # axs[index][0].set_xlim((400, row_size-500))
    axs[index][0].set_xlabel("Pixel position")
    axs[index][0].set_ylabel("Pixel intensity")
    axs[index][0].legend()
    
plt.savefig(os.path.join(registration_dir, "registration_results-profiles.pdf"), bbox_inches = 'tight', dpi=DPI)
plt.savefig(os.path.join(registration_dir, "registration_results-profiles.png"), bbox_inches = 'tight', dpi=DPI)

### 5.4 Show the evolution of the objective function

In [ ]:
plt.figure()
plt.title("Evolution of the fitness value of the best individual")
plt.ylabel("RMSE")
plt.xlabel("Number of evaluations")
plt.plot(fitness_set[:, 0], fitness_set[:, 1])
plt.show()

## 7. Perform a simulation of the CT scan acquisition

### 7.1 Apply the results of the optimisation

We apply the geometrical transformations on the 3D surface. We also use the optimised material composition.

In [ ]:
# gvxr.setMixture(sample_label, elements, default_weights)
# gvxr.setDensity(sample_label, default_density, "g/cm3")

# # Sample position on turntable
# gvxr.translateNode(sample_label, x_current[0], x_current[1], x_current[2], "mm")

# gvxr.rotateNode(sample_label, x_current[3], 0, 1, 0)
# gvxr.rotateNode(sample_label, x_current[4], 1, 0, 0)
# gvxr.rotateNode(sample_label, x_current[5], 0, 0, 1)

# gvxr.applyCurrentLocalTransformation(sample_label)

# mesh_fname = os.path.join(mesh_dir, sample_label + "-transformed.stl")
# gvxr.saveSTLfile(sample_label, mesh_fname);
# print("New mesh saved in:", mesh_fname);

### 7.2 Perform the CT scan acquisition

In [ ]:
JSON_fname = os.path.join(registration_dir, "CT-acquisition-real_part_model.json")
proj_path = os.path.join(registration_dir, "simulated_projections-real_part_model")

simulated_projections = False;
if  not os.path.exists(JSON_fname) or not os.path.exists(proj_path):
    
    if not os.path.exists(proj_path):
        os.mkdir(proj_path);
        
    total_incident_energy = gvxr.getTotalEnergyWithDetectorResponse()

    for i, (source_position, detector_position, detector_direction_x, detector_direction_y) in enumerate(zip(source_position_set, detector_position_set, detector_direction_x_set, detector_direction_y_set)):

        gvxr.setSourcePosition(source_position[0], source_position[1], source_position[2], "mm");

        gvxr.autoAlignDetector(False);
        gvxr.setDetectorPosition(detector_position[0], detector_position[1], detector_position[2], "mm");
        gvxr.setDetectorUpVector(detector_direction_y[0], detector_direction_y[1], detector_direction_y[2]);
        gvxr.setDetectorRightVector(detector_direction_x[0], detector_direction_x[1], detector_direction_x[2]);

        # Compute an X-ray image
        # We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.
        xray_image = np.array(gvxr.computeXRayImage(), dtype=np.single) / total_incident_energy
        imwrite(os.path.join(proj_path, "img_" + str(i).zfill(5) + ".tif"), xray_image, compression ="zlib");
        
    simulated_projections = True;

### 7.3 Save the corresponding JSON file

In [ ]:
if simulated_projections:
    gvxr2json.saveJSON(JSON_fname)

## 8. CT reconstruction with CIL

In [ ]:
# Import packages
from cil.io import TIFFWriter
from cil.processors import TransmissionAbsorptionConverter, Slicer
# from cil.recon import FDK
from cil.plugins.astra import FBP

from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer

from DTHEDataReader import *
from gvxrPython3.JSON2gVXRDataReader import JSON2gVXRDataReader

### 8.1 Load the simulated projections

In [ ]:
# Define the path where TIFF files will be saved
output_path = "recons-real_part_model";
save_path = os.path.join(registration_dir, output_path);

if not os.path.exists(save_path):

    # Instanciate the reader
    # reader = JSON2gVXRDataReader(file_name=JSON_fname);
    reader = DTHEDataReader(directory_path=data_path,
                            normalise=False,
                            tiff_directory_path=proj_path);

    # Read the data
    data = reader.read();

### 8.2 Inspect the geometry

Checkout what the data looks like.

In [ ]:
if not os.path.exists(save_path):
    print(data);

Checkout what the acquisition geometry looks like.

In [ ]:
if not os.path.exists(save_path):
    print(data.geometry);

CIL can even plot what the geometry looks like.

In [ ]:
# Plot and save the geometry
fname = os.path.join(registration_dir, "CT-acquisition-CAD_model-geometry.png");
# show_geometry(data.geometry).save(fname);

### 8.3 Apply the minus log

In [ ]:
if not os.path.exists(save_path):
    data_corr = TransmissionAbsorptionConverter(min_intensity=0.00001, white_level=data.max())(data);

In [ ]:
if not os.path.exists(save_path):
    # Release memory
    del data;

Inspect the projections

### 8.4 Define a ROI to save resources

The inspection performed above showed that there is a lot of air around the object. There is no need to waste resources and time reconstructing air.

In [ ]:
scaling_factor = 1;
roi_size = [800, 800, 2500];

### 8.5 Reconstruct the CT volume with CIL

In [ ]:
if not os.path.exists(save_path):
    start = time.time();

    # Prepare the data for Tigre
    data_corr.reorder(order='astra');

    # Reconstruct using FDK
    ig = data_corr.geometry.get_ImageGeometry();

    # Change the number of voxels in the reconstructed volume if a ROI is in use
    if roi_size is not None:
        ig.voxel_num_x = roi_size[0];
        ig.voxel_num_y = roi_size[1];
        ig.voxel_num_z = roi_size[2];

    # Apply a scaling factor if binning is in use
    if scaling_factor != 1:
        ig.voxel_num_x = round(ig.voxel_num_x // scaling_factor);
        ig.voxel_num_y = round(ig.voxel_num_y // scaling_factor);
        ig.voxel_num_z = round(ig.voxel_num_z // scaling_factor);

        ig.voxel_size_x *= scaling_factor;
        ig.voxel_size_y *= scaling_factor;
        ig.voxel_size_z *= scaling_factor;

        ig.voxel_size_x = np.max([ig.voxel_size_x, ig.voxel_size_y, ig.voxel_size_z]);
        ig.voxel_size_y = ig.voxel_size_z = ig.voxel_size_x;

    # Show the reconstructed volume voxel size
    print("Voxel size:", ig.voxel_size_x, ig.voxel_size_y, ig.voxel_size_z);

    # Instantiate the reconstruction algorithm
    fdk = FBP(ig, data_corr.geometry)
    fdk.set_input(data_corr)

    # Perform the actual CT reconstruction
    recons_FDK_cil = fdk.get_output();

    stop = time.time();
    # runtime, unit = getRuntime(start, stop);
    print("Execution time:", "{0:0.2f}".format(runtime), unit);

In [ ]:
if not os.path.exists(save_path):
    # Release memory
    del data_corr;

### 8.6 Save the reconstructed CT volume to disk for example as a stack of TIFFs

In [ ]:
if not os.path.exists(save_path):
    print("Print the CT data will be saved in:", save_path);   
    TIFFWriter(data=recons_FDK_cil, file_name=os.path.join(save_path, "out")).write();
    
    del recons_FDK_cil;

## 9.Voxelisation of the 3D object in the same volume space as the CT scan

### 9.1 Use a monochromatic beam

Overwrite the beam spectrum as there is no need to use polychromatism.

In [ ]:
incident_energy = 80
gvxr.setMonoChromatic(incident_energy, "keV", 1);

### 9.2 Use a constant for the linear attenuation coefficient of the sample, e.g. &mu; = 0.1 cm<sup>-1</sup>

In [ ]:
gvxr.setLinearAttenuationCoefficient(sample_label, 0.1, "cm-1")

### 9.3 Perform a CT scan acquisition in the same geometrical conditions

In [ ]:
proj_path = os.path.join(data_path, "simulated_projections-real_part_model-voxelisation")

if not os.path.exists(proj_path):
    
    if not os.path.exists(proj_path):
        os.mkdir(proj_path);
        
    total_incident_energy = gvxr.getTotalEnergyWithDetectorResponse()

    for i, (source_position, detector_position, detector_direction_x, detector_direction_y) in enumerate(zip(source_position_set, detector_position_set, detector_direction_x_set, detector_direction_y_set)):

        gvxr.setSourcePosition(source_position[0], source_position[1], source_position[2], "mm");

        gvxr.autoAlignDetector(False);
        gvxr.setDetectorPosition(detector_position[0], detector_position[1], detector_position[2], "mm");
        gvxr.setDetectorUpVector(detector_direction_y[0], detector_direction_y[1], detector_direction_y[2]);
        gvxr.setDetectorRightVector(detector_direction_x[0], detector_direction_x[1], detector_direction_x[2]);

        # Compute an X-ray image
        # We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.
        xray_image = np.array(gvxr.computeXRayImage(), dtype=np.single) / total_incident_energy
        imwrite(os.path.join(proj_path, "img_" + str(i).zfill(5) + ".tif"), xray_image, compression ="zlib");

## 10. Stop the simulation engine

In [ ]:
gvxr.terminate()

## 11. CT reconstruction with CIL   

### 11.1 Load the simulated projections

In [ ]:
# Define the path where TIFF files will be saved
output_path = "recons-real_part_model-voxelisation";
save_base_path = os.getcwd();
save_path = os.path.join(data_path, output_path);

In [ ]:
if not os.path.exists(save_path):
    reader = DTHEDataReader(directory_path=data_path,
                            normalise=False,
                            tiff_directory_path=os.path.join(data_path, "simulated_projections-real_part_model-voxelisation"));

    # Read the data
    data_original = reader.read();

### 11.2 Apply the minus log

In [ ]:
if not os.path.exists(save_path):
    data_absorption = -np.log(data_original)
    del data_original;

### 11.3 Define a ROI to save resources
### 11.4 Reconstruct the CT volume with CIL

In [ ]:
if not os.path.exists(save_path):
    
    start = time.time();

    # Prepare the data for Tigre
    data_absorption.reorder(order='astra');

    # Reconstruct using FDK
    ig = data_absorption.geometry.get_ImageGeometry();

    # Change the number of voxels in the reconstructed volume if a ROI is in use
    if roi_size is not None:
        ig.voxel_num_x = roi_size[0];
        ig.voxel_num_y = roi_size[1];
        ig.voxel_num_z = roi_size[2];

    # Apply a scaling factor if binning is in use
    if scaling_factor != 1:
        ig.voxel_num_x = round(ig.voxel_num_x // scaling_factor);
        ig.voxel_num_y = round(ig.voxel_num_y // scaling_factor);
        ig.voxel_num_z = round(ig.voxel_num_z // scaling_factor);

        ig.voxel_size_x *= scaling_factor;
        ig.voxel_size_y *= scaling_factor;
        ig.voxel_size_z *= scaling_factor;

        ig.voxel_size_x = np.max([ig.voxel_size_x, ig.voxel_size_y, ig.voxel_size_z]);
        ig.voxel_size_y = ig.voxel_size_z = ig.voxel_size_x;

    # Show the reconstructed volume voxel size
    print("Voxel size:", ig.voxel_size_x, ig.voxel_size_y, ig.voxel_size_z);

    # Instantiate the reconstruction algorithm
    fdk = FBP(ig, data_absorption.geometry)
    fdk.set_input(data_absorption)

    # Perform the actual CT reconstruction
    recons_FDK_cil = fdk.get_output();

    stop = time.time();
    runtime, unit = getRuntime(start, stop);
    print("Execution time:", "{0:0.2f}".format(runtime), unit);

### 11.5 Binarise the valume and save it to disk for example as a stack of TIFFs

In [ ]:
import SimpleITK as sitk

In [ ]:
if not os.path.exists(save_path):
    recons_array = np.round((recons_FDK_cil * 1000).as_array()).astype(np.uint8);

    sitk_img = sitk.GetImageFromArray(recons_array)
    median_filter = sitk.MedianImageFilter()
    median_filter.SetRadius(1)
    sitk_img = median_filter.Execute(sitk_img)

    recons_array = sitk.GetArrayFromImage(sitk_img)
    
    recons_array[0,:,:] = 0
    recons_array[ig.voxel_num_z - 1,:,:] = 0
    recons_array[recons_array>=200] = 0
    recons_array[recons_array>=5] = 255

    print("Print the CT data will be saved in:", save_path);
    
    os.mkdir(save_path);
    
    for i, img in enumerate(recons_array):

        imwrite(os.path.join(save_path, "slice_idx_" + str(i).zfill(4) + ".tiff"), img, compression ='zlib')    